# 自动摘要
有监督摘要---事先有一个做好的人工摘要，让自己的摘要与人工摘要尽量契合
无监督摘要---只有原文档，通过算法生成一个摘要
按照标点符号或分段符号，进行分句或分段。“。 ！ ？”
找出那些包含信息最多的句/段，将其合并起来作为原文档的摘要
信息----关键词，与文章主题相似度
或者提取出其中所隐含的信息，用其生成文档摘要----自动写作


自动摘要的基本实现思路：抽取式
直接提取文档中已存在的关键词、句子形成摘要
核心假设：一篇文档的中心思想可以用文档中的一句话/关键词或几句话/关键词加以概括

各句打分：目前算法最大的局别
基于统计：如TextTeaser，基于长度、位置、关键词等信息进行评分
基于网络图：如TextRank，按照节点权重进行评分
基于潜在语义：使用主题模型等，挖掘词句隐藏信息进行评分

句子选择
句子重组：保留前后语序，以保证摘要的可读性

分析思路1
    计算各分句和文章之间的余弦相似度
    将相似度最高的分句（或是最高的前几句合并起来）最为文章的摘要
    
分析思路2
    携带信息量的大小可以直接用包好的关键词数量的多少来代表
    使用TF-IDF提取文章的关键词
    实现方式
        1.找出包含关键词数量最多的句子（或者最多的几句并起来）作为摘要
        2.找出这些关键词首先出现的句子，将其合并起来作为摘要
        
自动摘要的其他思路：
压缩式：
    在抽取句子的同时，对这些句子进行信息压缩和融合
    能够有效提高单位摘要长度中的信息量，但会牺牲句子的可读性
    
生成式：
    直接从语义表达生成摘要句
    最接近摘要的本质，但实现难度很大
    目前尚未达到实用程度
    
自动摘要的效果评价：人工评价
    评价结果可信
    但成本高、效率低
    
Edmundson评价：（本质-人工评价+自动生成）
    是内部评价方法的一种（提供参考只要用于评价）
    根据专家摘要和自动摘要的句子重合率来对效果进行评价
    重合率p = 匹配句子数/专家文摘句子数 * 100%
    多个专家独立评价时，计算重合率的平均值
    要求：专家只能抽取句子形成摘要，且前后顺序保持不变
    
自动摘要的效果评价：ROUGE评价
一种自动摘要评价方法
基本思想：将系统生成的自动摘要与人工生成的标准摘要相对比，通过统计二者之间重叠的基本单元（n元语法、词序列和词对）数目，来评价摘要的质量
    计算原始文档和摘要n-gram共现信息作为评价标准
    
包括一系列方法：ROUGE-N(N=1~4),-L，-S，-W，-SU等

ROUGE-N简介
......

优点：直观、简洁、能反应词序
缺点：区分度不高（好坏摘要）

ROUGE-L：寻找最长的公共子序列
ROUGE-W：为连续匹配比不连续匹配赋予更大的权重，从而比LCS区分度高
ROUGE-S：考虑了所有按词序排列的词对，比n-gram模型更能深入反应句子级词序


# 以句子为单位生成自动摘要

In [35]:
import pandas as pd
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()


#切分句子
def cut_sentence(intxt):  
    delimiters = frozenset('。！？')  #以这三个符号作为分句标识，集合
    buf = []  #缓冲list
    for ch in intxt:  
        buf.append(ch)  
        if delimiters.__contains__(ch):  
            yield ''.join(buf)  
            buf = []  
    if buf:  
        yield ''.join(buf)  

sentdf = pd.DataFrame(cut_sentence(df.content[51]))

sentdf#数据框

# 去除过短的句子，避免摘要出现无意义的内容
sentdf['txtlen'] = sentdf[0].apply(len)#计算每个记录的长度
sentdf.head()
sentlist = sentdf[0][sentdf.txtlen > 20]#选出长度大于20的
print(len(sentlist))
sentlist#数据框

#------------------要原始素材转换为矩阵-----------------
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import jieba
txtlist = [ " ".join(jieba.lcut(w)) for w in sentlist]

vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(txtlist) # 将文本中的词语转换为词频矩阵
tfidf_matrix = TfidfTransformer().fit_transform(X)  #转成TF-IDF矩阵
print(tfidf_matrix.todense)
print(tfidf_matrix.todense().shape)
#--------------------------使用pagerank算法--------------
# 利用nx包实现pagerank算法
import networkx as nx  

similarity = nx.from_scipy_sparse_matrix(tfidf_matrix * tfidf_matrix.T)  #计算响应相似矩阵-----IF-IDF矩阵*它的转置，相当于提取出了特征根
scores = nx.pagerank(similarity) 
scores #每句话的一个pagerank评分
#找出评分高的矩阵
tops = sorted(scores.items(), key = lambda x: x[1], reverse = True)#排序
tops

topn = 3
topsent = sorted(tops[:topn])
abstract = ''
for item in topsent:
    abstract = abstract + sentlist.iloc[item[0]] +"......"
abstract[:-6]

47
<bound method spmatrix.todense of <47x382 sparse matrix of type '<class 'numpy.float64'>'
	with 735 stored elements in Compressed Sparse Row format>>
(47, 382)


'三、基于大数据时代背景，计算机技术的核心程序（一）计算机信息采集技术计算机采集技术属于计算机处理信息的首要步骤，受到专业技术人员的重视。......另外，相关技术人员要以主流市场需求方向为基准，进一步优化完善计算机信息安全技术、信息传播技术、信息存储技术与信息采集技术。......为此，在大数据时代背景下，相关技术人员要加大计算机技术研究深度，以信息安全技术、信息传播技术、信息采集技术以及信息存储技术为切入点，不断提高信息处理效率与精确性，全面保护用户财产安全与人身安全。'

# 读取文章中有的摘要

In [14]:
def read_part(filename):
    content = []
    recording = False
    with open(filename,encoding='utf-8',errors='ignore') as f:
        for line in f:
            line = line.strip('\n')
            #print(line)
            if line.find('关键词') != -1:
                #print('q')
                break
            if line.find('摘 要') != -1 or line.find('摘要') != -1 or line.find('摘') != -1:
                recording = True
            if recording:
                content.append(line)
                #print(content)
    return ''.join(content)
contents = read_part(r'C:\EdmsData\txt\试析大数据时代计算机技术的应用_孙振军.txt')
print(contents)

摘要：进入“互联网+”时代，计算机性能日趋完善，并逐步拓展应用到各个行业领域。但是，在应用计算机技术的过程中，随着数据量的膨胀与数据类型的复杂化，如何提高计算机技术的信息处理效率，保证用户信息安全，成为业内人士研究的关键。由此，本文基于大数据时代背景，围绕计算机技术的实践应用展开探究。


In [16]:
# -*- coding: utf-8 -*-
"""
Created on Sun May  5 18:12:03 2019

@author: Administrator
"""
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import jieba
import networkx as nx 
from MyDataFrame import MyDataFrame

class GetAbstract(object):
    def readAbstract(self,filename):
        filepath = 'C:/EdmsData/txt/'+filename+'.txt'
        content = []
        recording = False
        with open(filepath,encoding='utf-8',errors='ignore') as f:
            for line in f:
                line = line.strip('\n')
                #print(line)
                if line.find('关键词') != -1:
                    #print('q')
                    break
                if line.find('摘 要') != -1 or line.find('摘要') != -1 or line.find('摘'+ '\n' +'要') != -1:
                    recording = True
                if recording:
                    content.append(line)
                    #print(content)
        return ''.join(content)


    def generateAbstract(self,content):
        #1.形成数据框
        ga = GetAbstract()
        sentdf = pd.DataFrame(ga.cut_sentence(content))
        # 去除过短的句子，避免摘要出现无意义的内容
        sentdf['txtlen'] = sentdf[0].apply(len)#计算每个记录的长度
        #sentdf.head()
        sentlist = sentdf[0][sentdf.txtlen > 20]#选出长度大于20的
        print(len(sentlist))
        
        txtlist = [ " ".join(jieba.lcut(w)) for w in sentlist]
        vectorizer = CountVectorizer() 
        X = vectorizer.fit_transform(txtlist) # 将文本中的词语转换为词频矩阵
        tfidf_matrix = TfidfTransformer().fit_transform(X)  #转成TF-IDF矩阵
        
        similarity = nx.from_scipy_sparse_matrix(tfidf_matrix * tfidf_matrix.T)  #计算相应相似矩阵-----IF-IDF矩阵*它的转置，相当于提取出了特征根
        scores = nx.pagerank(similarity) 
        #print(scores) #每句话的一个pagerank评分
        #找出评分高的矩阵
        tops = sorted(scores.items(), key = lambda x: x[1], reverse = True)#排序
        print(tops)
      
        topn = 3  #5句话
        topsent = sorted(tops[:topn])
        abstract = ''
        for item in topsent:
            abstract = abstract + sentlist.iloc[item[0]] + '\n'
        abstract_str = abstract[:-6]
        
        return abstract_str
        
        

    def cut_sentence(self,intxt):  
        delimiters = frozenset('。！？')  #以这三个符号作为分句标识，集合
        buf = []  #缓冲list
        for ch in intxt:  
            buf.append(ch)  
            if delimiters.__contains__(ch):  
                yield ''.join(buf)  
                buf = []  
        if buf:  
            yield ''.join(buf)  
            
        
if __name__ == '__main__':
    ga = GetAbstract()
    abstract_contents = ga.readAbstract('电力文本数据挖掘现状及挑战_王慧芳')
    print(abstract_contents)
    mdf = MyDataFrame()
    df = mdf.new_DataFrame()
    abstract_contents2 = ga.generateAbstract(df.content[2])
    print(abstract_contents2)

摘 要 ：文本数据是电力大数据的重要组成部分对其进行有效挖掘是智能电网深入全面发展的需要在目前已有研究成果的基础上对电力领域文本数据挖掘的现状及挑战进行了深入剖析首先分析了文本挖掘技术的发展过程及存在难题接着重点分析了电力文本数据挖掘的关键技术及其研究现状包括文本预处理技术文本表示方法以及数据挖掘方法然后以电力设备缺陷文本为对象介绍了文本挖掘技术在电力领域的应用包括缺陷文本质量的提升与保证缺陷文本严重程度自动 分类缺陷发生部件及程度的自动提取缺陷文本检索基于缺陷文本的电力设备健康状态评价等可为其他类型电力文本的挖掘提供参考最后探讨了电力文本挖掘面临的挑战以及未来发展方向电力大数据文本数据数据挖掘自然语言处理电力设备缺陷文本
38
[(23, 0.039025894197919353), (2, 0.035874301237116595), (12, 0.034718507163513074), (18, 0.033893428041901423), (10, 0.033820875627797516), (27, 0.033726583770799491), (14, 0.033187706373575473), (5, 0.032305145402210224), (7, 0.032139363975505986), (29, 0.031374707844925631), (21, 0.030997761788435226), (17, 0.030369369531946449), (4, 0.029772271678544331), (22, 0.028865689492708478), (13, 0.028252989741466273), (16, 0.02820818956599741), (32, 0.02728198240352556), (0, 0.027078271985974456), (25, 0.026676451907802979), (19, 0.026493555331011616), (3, 0.025994146903442771), (30, 0.025548997055915016), (1, 0.02534558154801432), (28, 0.025323532265643635), (26, 0.02443837623

# TextRank

In [33]:
import pandas as pd
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()
for i in range(len(df)):
    print(df.filename[i] + "----"+str(i))


_垃圾清理挑战_传播环保正能量_叱狼----0
中国特色政治经济学理论说服力的重要源泉_武力----1
中小民营企业融资存在的困难和解决对策_陶永珉----2
习近平的人格魅力及其对马克思主义大众化的推动_钟敏丰----3
云计算视域下数据挖掘技术_杨继武----4
企业财务风险的识别与内部控制对策_郭涛----5
优化营商环境__为民营经济发展创造良好条件_金辉----6
体育总局举行党组理论学习中心组_扩大_学习会议_顾宁----7
关于蒙德里安艺术风格的思考_袁可----8
关于计算机人工智能技术的发展分析与应用探讨_聂华----9
农业技术推广对提升农业种植业的作用分析_郭晓芳----10
卫星数据挖掘节点级并行与优化方法_鲍军鹏----11
印象派艺术特色变革浅析_王亚萍----12
国外对计算机软件知识产权的保护_陈红----13
基于_大数据_时代下计算机信息处理技术研究_周琪栋----14
基于大数据的商业智能在电商数据分析中的应用_钱丹丹----15
大数据时代的计算机网络安全及防范技术_欧东鑫----16
大数据时代计算机网络信息安全问题研究_亓兵----17
妙在_藏_与_不藏_之间_由花鼓小戏_老四碰车_说开去_谭孝红----18
宏观经济统计分析发展的几点认识_杨曹正----19
对农业种植技术和现代农业机械化的相关性探讨_顾佳宝----20
广州严把污染源普查质量控制关口_陈昊----21
我国农业机械化发展形势与对策分析_苗唤维----22
我国数字经济高质量发展的路径研究_徐鹏远_苑博_冯晓琳----23
我国设施蔬菜科技与产业发展现状及趋势_苗艳香----24
探索有机农业可持续保护发展的核心问题_赵庆杰----25
探讨新形势下投融资项目管理模式及其改进_孙萱----26
改革为中国羽毛球积蓄能量_周圆----27
数字经济新时代下的转型与升级----28
数据挖掘技术在电子商务管理中的应用_高宇----29
数据挖掘技术在电网运营监控平台建设中的应用_李呓瑾----30
数据挖掘技术在网络营销中的应用分析_王冬屏----31
文本挖掘与中文文本挖掘模型研究----32
新时代习近平党的思想建设理论溯源及其新内涵_尚杨----33
梅州_村庄保洁全覆盖__人居环境大改善_钟奇振----34
浅谈农业技术推广的对策_张奎栋----3